# TF

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ["TF_FORCE_GPU_ALLOW_GROWTH"] = "true"

In [2]:
import tensorflow as tf
import tensorflow.keras.backend as K

import sys
sys.path.append(r'C:\Users\81908\jupyter_notebook\poetry_work\tfgpu\atmaCup_#8\notebook\tabnet')
from tabnet_tf import TabNetRegressor, StackedTabNetRegressor

Tensorflow version 2.3.1


In [3]:
from adabelief_tf import AdaBeliefOptimizer

In [4]:
import tensorflow as tf


def build_callbacks(
    model_path, factor=0.1, mode="auto", monitor="val_loss", patience=0, verbose=0
):
    early_stopping = tf.keras.callbacks.EarlyStopping(
        mode=mode, monitor=monitor, patience=patience, verbose=verbose
    )
    model_checkpoint = tf.keras.callbacks.ModelCheckpoint(
        model_path, mode=mode, monitor=monitor, save_best_only=True, verbose=verbose
    )
    reduce_lr_on_plateau = tf.keras.callbacks.ReduceLROnPlateau(
        factor=factor, monitor=monitor, mode=mode, verbose=verbose
    )

    return [early_stopping, model_checkpoint, reduce_lr_on_plateau]

# base

In [5]:
import os
import gc
import re
import math
import pickle
import joblib
import warnings

import numpy as np
import pandas as pd
from tqdm.notebook import tqdm

from sklearn.model_selection import KFold, GroupKFold

import lightgbm as lgb

warnings.simplefilter("ignore")
pd.set_option('display.max_columns', None)

In [6]:
import random as rn
import numpy as np


def set_seed(seed=0):
    os.environ["PYTHONHASHSEED"] = str(seed)

    rn.seed(seed)
    np.random.seed(seed)

In [7]:
from sklearn.metrics import mean_squared_log_error


def score(y, y_pred):
    RMSLE = np.sqrt(np.mean(((np.log(y + 1) - np.log(y_pred + 1)) ** 2)))
    #RMSLE = mean_squared_log_error(y, y_pred) ** 0.5
    return RMSLE

# Data load

In [8]:
import pandas as pd

DATADIR = r"C:\Users\81908\jupyter_notebook\poetry_work\tfgpu\atmaCup_#8\data\atmacup08-dataset"
train = pd.read_csv(f"{DATADIR}/train.csv")
test = pd.read_csv(f"{DATADIR}/test.csv")
df = pd.concat([train, test], axis=0)

# 前処理

In [9]:
import numpy as np

# tbd(確認中)を欠損にする
df["User_Score"] = df["User_Score"].replace("tbd", np.nan)

In [10]:
# -1で補完
cate_cols = [
    "Name",
    "Platform",
    "Year_of_Release",
    "Genre",
    "Publisher",
    "Developer",
    "Rating",
]
for col in cate_cols:
    df[col].fillna(-1, inplace=True)

In [11]:
def impute_null_add_flag_col(
    df, strategy="mean", cols_with_missing=None, fill_value=None
):
    """欠損値を補間して欠損フラグ列を追加する
    fill_value はstrategy="constant"の時のみ有効になる補間する定数
    """
    from sklearn.impute import SimpleImputer

    df_plus = df.copy()

    for col in cols_with_missing:
        # 欠損フラグ列を追加
        df_plus[col + "_was_missing"] = df[col].isnull()
        df_plus[col + "_was_missing"] = df_plus[col + "_was_missing"].astype(int)
        # 欠損値を平均値で補間
        my_imputer = SimpleImputer(strategy=strategy, fill_value=fill_value)
        df_plus[col] = my_imputer.fit_transform(df[[col]])

    return df_plus


df = impute_null_add_flag_col(
    df, strategy="most_frequent", cols_with_missing=["User_Score"]
)  # 最頻値で補間

df = impute_null_add_flag_col(
    df,
    cols_with_missing=[
        "Critic_Score",
        "Critic_Count",
        "User_Count",
    ],
)  # 平均値で補間（数値列のみ）

In [12]:
# User_Scoreを数値列にする
df["User_Score"] = df["User_Score"].astype("float")

# User_Scoreを文字列にする
df["Year_of_Release"] = df["Year_of_Release"].astype("str")

In [13]:
# ラベルエンコディング
cate_cols = df.select_dtypes(include=["object", "category", "bool"]).columns.to_list()
for col in cate_cols:
    df[col], uni = pd.factorize(df[col])

In [14]:
train = df.iloc[: train.shape[0]]
test = df.iloc[train.shape[0] :].reset_index(drop=True)

# 目的変数
sales_cols = [
    "NA_Sales",
    "EU_Sales",
    "JP_Sales",
    "Other_Sales",
    "Global_Sales",
]

train_drop_sales = train.drop(sales_cols, axis=1)
test = test.drop(sales_cols, axis=1)

In [15]:
# 欠損データ確認
_df = pd.DataFrame({"is_null": df.isnull().sum()})
_df[_df["is_null"] > 0]

,is_null
NA_Sales,8360
EU_Sales,8360
JP_Sales,8360
Other_Sales,8360
Global_Sales,8360


# FE

In [16]:
df = pd.concat([train_drop_sales, test], axis=0)

In [17]:
# 行単位で統計量とる
def add_num_row_agg(df_all, agg_num_cols):
    """行単位の統計量列追加
    agg_num_cols は数値列だけでないとエラー"""
    import warnings

    warnings.filterwarnings("ignore")

    df = df_all[agg_num_cols]
    cols = df.columns.to_list()
    cols = map(str, cols)  # 文字列にする
    col_name = "_".join(cols)

    df_all[f"row_{col_name}_sum"] = df.sum(axis=1)
    df_all[f"row_{col_name}_mean"] = df.mean(axis=1)
    df_all[f"row_{col_name}_std"] = df.std(axis=1)
    # df_all[f"row_{col_name}_skew"] = df.skew(axis=1)  # 歪度  # nanになるから入れない
    
    return df_all

    
df = add_num_row_agg(df, ["Critic_Score", "User_Score"])
df = add_num_row_agg(df, ["Critic_Count", "User_Count"])

In [18]:
import numpy as np
import pandas as pd


# A列でグループして集計したB列は意味がありそうと仮説たててから統計値列作ること
# 目的変数をキーにして集計するとリークしたターゲットエンコーディングになるため説明変数同士で行うこと
def grouping(df, cols, agg_dict, prefix=""):
    """特定のカラムについてgroup化された特徴量の作成を行う
    Args:
        df (pd.DataFrame): 特徴量作成のもととなるdataframe
        cols (str or list): group by処理のkeyとなるカラム (listで複数指定可能)
        agg_dict (dict): 特徴量作成を行いたいカラム/集計方法を指定するdictionary
        prefix (str): 集約後のカラムに付与するprefix name

    Returns:
        df (pd.DataFrame): 特定のカラムについてgroup化された特徴量群
    """
    group_df = df.groupby(cols).agg(agg_dict)
    group_df.columns = [prefix + c[0] + "_" + c[1] for c in list(group_df.columns)]
    group_df.reset_index(inplace=True)

    return group_df

class AggUtil():
    ############## カテゴリ列 vs. 数値列について ##############
    @staticmethod
    def percentile(n):
        """パーセンタイル"""
        def percentile_(x):
            return np.percentile(x, n)
        percentile_.__name__ = "percentile_%s" % n
        return percentile_

    @staticmethod
    def diff_percentile(n1, n2):
        """パーセンタイルの差"""
        def diff_percentile_(x):
            p1 = np.percentile(x, n1)
            p2 = np.percentile(x, n2)
            return p1 - p2
        diff_percentile_.__name__ = f"diff_percentile_{n1}-{n2}"
        return diff_percentile_

    @staticmethod
    def ratio_percentile(n1, n2):
        """パーセンタイルの比"""
        def ratio_percentile_(x):
            p1 = np.percentile(x, n1)
            p2 = np.percentile(x, n2)
            return p1 / p2
        ratio_percentile_.__name__ = f"ratio_percentile_{n1}-{n2}"
        return ratio_percentile_
    
    @staticmethod
    def mean_var():
        """平均分散"""
        def mean_var_(x):
            x = x.dropna()
            return np.std(x) / np.mean(x)
        mean_var_.__name__ = f"mean_var"
        return mean_var_
    
    @staticmethod
    def diff_mean():
        """平均との差の中央値(aggは集計値でないとエラーになるから中央値をとる)"""
        def diff_mean_(x):
            x = x.dropna()
            return np.median(x - np.mean(x))
        diff_mean_.__name__ = f"diff_mean"
        return diff_mean_
    
    @staticmethod
    def ratio_mean():
        """平均との比の中央値(aggは一意な値でないとエラーになるから中央値をとる)"""
        def ratio_mean_(x):
            x = x.dropna()
            return np.median(x / np.mean(x))
        ratio_mean_.__name__ = f"ratio_mean"
        return ratio_mean_
    
    @staticmethod
    def hl_ratio():
        """平均より高いサンプル数と低いサンプル数の比率"""
        def hl_ratio_(x):
            x = x.dropna()
            n_high = x[x >= np.mean(x)].shape[0]
            n_low = x[x < np.mean(x)].shape[0]
            if n_low == 0:
                return 1.0
            else:
                return n_high / n_low
        hl_ratio_.__name__ = f"hl_ratio"
        return hl_ratio_
    
    @staticmethod
    def ratio_range():
        """最大/最小"""
        def ratio_range_(x):
            x = x.dropna()
            if np.min(x) == 0:
                return 1.0
            else:
                return np.max(x) / np.min(x)
        ratio_range_.__name__ = f"ratio_range"
        return ratio_range_
    
    @staticmethod
    def beyond1std():
        """1stdを超える比率"""
        def beyond1std_(x):
            x = x.dropna()
            return x[np.abs(x) > np.abs(np.std(x))].shape[0] / x.shape[0]
        beyond1std_.__name__ = "beyond1std"
        return beyond1std_
    
    @staticmethod
    def zscore():
        """Zスコアの中央値(aggは一意な値でないとエラーになるから中央値をとる)"""
        def zscore_(x):
            x = x.dropna()
            return np.median((x - np.mean(x)) / np.std(x))
        zscore_.__name__ = "zscore"
        return zscore_
    ######################################################
    
    ############## カテゴリ列 vs. カテゴリ列について ##############
    @staticmethod
    def freq_entropy():
        """出現頻度のエントロピー"""
        from scipy.stats import entropy
        def freq_entropy_(x):
            return entropy(x.value_counts().values)
        freq_entropy_.__name__ = "freq_entropy"
        return freq_entropy_
    
    @staticmethod
    def freq1name():
        """最も頻繁に出現するカテゴリの数"""
        def freq1name_(x):
            return x.value_counts().sort_values(ascending=False)[0]
        freq1name_.__name__ = "freq1name"
        return freq1name_
    
    @staticmethod
    def freq1ratio():
        """最も頻繁に出現するカテゴリ/グループの数"""
        def freq1ratio_(x):
            frq = x.value_counts().sort_values(ascending=False)
            return frq[0] / frq.shape[0]
        freq1ratio_.__name__ = "freq1ratio"
        return freq1ratio_
    #########################################################


# 集計する数値列指定
value_agg = {
    "User_Count": [
        "max",
        "min",
        "mean",
        #"std",  # 標準偏差
        #"skew",  # 歪度
        #pd.DataFrame.kurt,  # 尖度
    ],
    "Critic_Count": [
        "max",
        "min",
        "mean",
        #"std",  # 標準偏差
        #"skew",  # 歪度
        #pd.DataFrame.kurt,  # 尖度
    ],
    "User_Score": [
        "max",
        "min",
        "mean",
        #"std",  # 標準偏差
        #"skew",  # 歪度
        #pd.DataFrame.kurt,  # 尖度
    ],
    "Critic_Score": [
        "max",
        "min",
        "mean",
        #"std",  # 標準偏差
        #"skew",  # 歪度
        #pd.DataFrame.kurt,  # 尖度
    ],
}
# グループ化するカテゴリ列でループ
for key in ["Platform", "Genre", "Publisher", "Developer", "Rating"]:
    feature_df = grouping(df, key, value_agg, prefix=key + "_")
    df = pd.merge(df, feature_df, how="left", on=key)

In [19]:
df["Critic_Score_*_Critic_Count"] = df["Critic_Score"] * df["Critic_Count"]
df["User_Score_*_User_Count"] = df["User_Score"] * df["User_Count"]
df["Critic_Score_*_User_Score"] = df["Critic_Score"] * df["User_Score"]
df["Critic_Count_*_User_Count"] = df["Critic_Count"] * df["User_Count"]
df["Critic_Count_+_User_Count"] = df["Critic_Count"] + df["User_Count"]
df["Critic_Count_-_User_Count"] = df["Critic_Count"] - df["User_Count"]
df["Critic_Count_/_all_Count"] = df["Critic_Count"] / df["Critic_Count_+_User_Count"]

In [20]:
## KMeansでクラスタリングした列追加
#
#from sklearn.cluster import KMeans
#
#def fe_cluster(df, kind, features, n_clusters=100, SEED=42, is_dummies=False):
#    df_ = df[features].copy()
#    kmeans_cells = KMeans(n_clusters=n_clusters, random_state=SEED).fit(df_)
#    df[f'clusters_{kind}'] = kmeans_cells.predict(df_.values)
#    df = pd.get_dummies(df, columns=[f'clusters_{kind}']) if is_dummies else df
#
#    return df
#
#df = fe_cluster(df, kind="cate_cols", 
#                features=["Name", "Genre", "Publisher", "Developer", "Rating"], 
#                n_clusters=3000)
#df = fe_cluster(df, kind="Score_Count", 
#                features=["Critic_Score", "User_Score", "Critic_Count", "User_Count"], 
#                n_clusters=300)

In [21]:
# 種類数少な目の列はダミー化して残す

float_cols = df.columns.to_list()

for col in ["Critic_Count_was_missing", "Critic_Score_was_missing", "User_Count_was_missing", "User_Score_was_missing"]:
    float_cols.remove(col)

df = pd.get_dummies(df, columns=[f'Rating'])
df = pd.get_dummies(df, columns=[f'Genre'])
df = pd.get_dummies(df, columns=[f'Platform'])

# 種類数少な目の列は消しとく
df = df.drop(["Name", "Year_of_Release", "Publisher", "Developer"], axis=1)

float_cols = np.intersect1d(np.array(float_cols), np.array(df.columns.to_list()))
float_cols

array(['Critic_Count', 'Critic_Count_*_User_Count',
       'Critic_Count_+_User_Count', 'Critic_Count_-_User_Count',
       'Critic_Count_/_all_Count', 'Critic_Score',
       'Critic_Score_*_Critic_Count', 'Critic_Score_*_User_Score',
       'Developer_Critic_Count_max', 'Developer_Critic_Count_mean',
       'Developer_Critic_Count_min', 'Developer_Critic_Score_max',
       'Developer_Critic_Score_mean', 'Developer_Critic_Score_min',
       'Developer_User_Count_max', 'Developer_User_Count_mean',
       'Developer_User_Count_min', 'Developer_User_Score_max',
       'Developer_User_Score_mean', 'Developer_User_Score_min',
       'Genre_Critic_Count_max', 'Genre_Critic_Count_mean',
       'Genre_Critic_Count_min', 'Genre_Critic_Score_max',
       'Genre_Critic_Score_mean', 'Genre_Critic_Score_min',
       'Genre_User_Count_max', 'Genre_User_Count_mean',
       'Genre_User_Count_min', 'Genre_User_Score_max',
       'Genre_User_Score_mean', 'Genre_User_Score_min',
       'Platform_Critic_C

In [22]:
## RankGauss
#from sklearn.preprocessing import QuantileTransformer
#
#_df = df[float_cols]
#
#qt = QuantileTransformer(n_quantiles=100, random_state=42, output_distribution="normal")
#df[_df.columns] = qt.fit_transform(_df)

In [23]:
# 欠損データ確認
_df = pd.DataFrame({"is_null": df.isnull().sum()})
_df[_df["is_null"] > 0]

,is_null


In [24]:
train_drop_sales = df.iloc[: train.shape[0]]
test = df.iloc[train.shape[0] :].reset_index(drop=True)

In [25]:
train_drop_sales

,Critic_Score,Critic_Count,User_Score,User_Count,User_Score_was_missing,Critic_Score_was_missing,Critic_Count_was_missing,User_Count_was_missing,row_Critic_Score_User_Score_sum,row_Critic_Score_User_Score_mean,row_Critic_Score_User_Score_std,row_Critic_Count_User_Count_sum,row_Critic_Count_User_Count_mean,row_Critic_Count_User_Count_std,Platform_User_Count_max,Platform_User_Count_min,Platform_User_Count_mean,Platform_Critic_Count_max,Platform_Critic_Count_min,Platform_Critic_Count_mean,Platform_User_Score_max,Platform_User_Score_min,Platform_User_Score_mean,Platform_Critic_Score_max,Platform_Critic_Score_min,Platform_Critic_Score_mean,Genre_User_Count_max,Genre_User_Count_min,Genre_User_Count_mean,Genre_Critic_Count_max,Genre_Critic_Count_min,Genre_Critic_Count_mean,Genre_User_Score_max,Genre_User_Score_min,Genre_User_Score_mean,Genre_Critic_Score_max,Genre_Critic_Score_min,Genre_Critic_Score_mean,Publisher_User_Count_max,Publisher_User_Count_min,Publisher_User_Count_mean,Publisher_Critic_Count_max,Publisher_Critic_Count_min,Publisher_Critic_Count_mean,Publisher_User_Score_max,Publisher_User_Score_min,Publisher_User_Score_mean,Publisher_Critic_Score_max,Publisher_Critic_Score_min,Publisher_Critic_Score_mean,Developer_User_Count_max,Developer_User_Count_min,Developer_User_Count_mean,Developer_Critic_Count_max,Developer_Critic_Count_min,Developer_Critic_Count_mean,Developer_User_Score_max,Developer_User_Score_min,Developer_User_Score_mean,Developer_Critic_Score_max,Developer_Critic_Score_min,Developer_Critic_Score_mean,Rating_User_Count_max,Rating_User_Count_min,Rating_User_Count_mean,Rating_Critic_Count_max,Rating_Critic_Count_min,Rating_Critic_Count_mean,Rating_User_Score_max,Rating_User_Score_min,Rating_User_Score_mean,Rating_Critic_Score_max,Rating_Critic_Score_min,Rating_Critic_Score_mean,Critic_Score_*_Critic_Count,User_Score_*_User_Count,Critic_Score_*_User_Score,Critic_Count_*_User_Count,Critic_Count_+_User_Count,Critic_Count_-_User_Count,Critic_Count_/_all_Count,Rating_0,Rating_1,Rating_2,Rating_3,Rating_4,Rating_5,Rating_6,Rating_7,Rating_8,Genre_0,Genre_1,Genre_2,Genre_3,Genre_4,Genre_5,Genre_6,Genre_7,Genre_8,Genre_9,Genre_10,Genre_11,Genre_12,Platform_0,Platform_1,Platform_2,Platform_3,Platform_4,Platform_5,Platform_6,Platform_7,Platform_8,Platform_9,Platform_10,Platform_11,Platform_12,Platform_13,Platform_14,Platform_15,Platform_16,Platform_17,Platform_18,Platform_19,Platform_20,Platform_21,Platform_22,Platform_23,Platform_24,Platform_25,Platform_26,Platform_27,Platform_28,Platform_29,Platform_30
0,74.000000,17.000000,7.9,22.000000,0,0,0,0,81.900000,40.950000,46.739758,39.000000,19.500000,3.535534,2147.000000,4.000000,115.942220,87.000000,4.000000,24.094851,9.3,0.2,7.351439,97.000000,19.000000,66.244882,8003.0,4.0,176.706842,106.0,4.0,27.157274,9.5,0.3,7.394926,98.0,19.0,67.656132,2679.000000,4.0,177.911378,89.0,4.0,25.355691,8.9,2.5,7.232340,96.0,24.0,69.427020,162.229908,4.0,48.974924,60.000000,4.0,21.126255,9.1,5.2,7.472727,86.0,51.000000,71.973186,5999.0,4.0,101.723715,89.0,4.0,23.408428,9.3,0.6,7.106056,95.0,19.0,67.229041,1258.000000,173.800000,584.600000,374.00000,39.000000,-5.000000,0.435897,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,78.000000,22.000000,6.6,28.000000,0,0,0,0,84.600000,42.300000,50.487424,50.000000,25.000000,4.242641,2147.000000,4.000000,115.942220,87.000000,4.000000,24.094851,9.3,0.2,7.351439,97.000000,19.000000,66.244882,8003.0,4.0,176.706842,106.0,4.0,27.157274,9.5,0.3,7.394926,98.0,19.0,67.656132,1509.000000,4.0,112.854538,74.0,4.0,24.819702,9.1,0.6,7.088889,93.0,43.0,69.938698,162.229908,4.0,48.974924,60.000000,4.0,21.126255,9.1,5.2,7.472727,86.0,51.000000,71.973186,5999.0,4.0,101.723715,89.0,4.0,23.408428,9.3,0.6,7.106056,95.0,19.0,67.229041,1716.000000,184.800000,514.800000,616.00000,50.000000,-6.000000,0.440000,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,73

In [26]:
test

,Critic_Score,Critic_Count,User_Score,User_Count,User_Score_was_missing,Critic_Score_was_missing,Critic_Count_was_missing,User_Count_was_missing,row_Critic_Score_User_Score_sum,row_Critic_Score_User_Score_mean,row_Critic_Score_User_Score_std,row_Critic_Count_User_Count_sum,row_Critic_Count_User_Count_mean,row_Critic_Count_User_Count_std,Platform_User_Count_max,Platform_User_Count_min,Platform_User_Count_mean,Platform_Critic_Count_max,Platform_Critic_Count_min,Platform_Critic_Count_mean,Platform_User_Score_max,Platform_User_Score_min,Platform_User_Score_mean,Platform_Critic_Score_max,Platform_Critic_Score_min,Platform_Critic_Score_mean,Genre_User_Count_max,Genre_User_Count_min,Genre_User_Count_mean,Genre_Critic_Count_max,Genre_Critic_Count_min,Genre_Critic_Count_mean,Genre_User_Score_max,Genre_User_Score_min,Genre_User_Score_mean,Genre_Critic_Score_max,Genre_Critic_Score_min,Genre_Critic_Score_mean,Publisher_User_Count_max,Publisher_User_Count_min,Publisher_User_Count_mean,Publisher_Critic_Count_max,Publisher_Critic_Count_min,Publisher_Critic_Count_mean,Publisher_User_Score_max,Publisher_User_Score_min,Publisher_User_Score_mean,Publisher_Critic_Score_max,Publisher_Critic_Score_min,Publisher_Critic_Score_mean,Developer_User_Count_max,Developer_User_Count_min,Developer_User_Count_mean,Developer_Critic_Count_max,Developer_Critic_Count_min,Developer_Critic_Count_mean,Developer_User_Score_max,Developer_User_Score_min,Developer_User_Score_mean,Developer_Critic_Score_max,Developer_Critic_Score_min,Developer_Critic_Score_mean,Rating_User_Count_max,Rating_User_Count_min,Rating_User_Count_mean,Rating_Critic_Count_max,Rating_Critic_Count_min,Rating_Critic_Count_mean,Rating_User_Score_max,Rating_User_Score_min,Rating_User_Score_mean,Rating_Critic_Score_max,Rating_Critic_Score_min,Rating_Critic_Score_mean,Critic_Score_*_Critic_Count,User_Score_*_User_Count,Critic_Score_*_User_Score,Critic_Count_*_User_Count,Critic_Count_+_User_Count,Critic_Count_-_User_Count,Critic_Count_/_all_Count,Rating_0,Rating_1,Rating_2,Rating_3,Rating_4,Rating_5,Rating_6,Rating_7,Rating_8,Genre_0,Genre_1,Genre_2,Genre_3,Genre_4,Genre_5,Genre_6,Genre_7,Genre_8,Genre_9,Genre_10,Genre_11,Genre_12,Platform_0,Platform_1,Platform_2,Platform_3,Platform_4,Platform_5,Platform_6,Platform_7,Platform_8,Platform_9,Platform_10,Platform_11,Platform_12,Platform_13,Platform_14,Platform_15,Platform_16,Platform_17,Platform_18,Platform_19,Platform_20,Platform_21,Platform_22,Platform_23,Platform_24,Platform_25,Platform_26,Platform_27,Platform_28,Platform_29,Platform_30
0,84.000000,23.000000,8.0,19.000000,0,0,0,0,92.000000,46.000000,53.740115,42.000000,21.000000,2.828427,1283.0,4.0,64.110095,91.000000,4.0,26.369807,9.3,0.5,7.587015,97.0,19.0,69.752185,8003.0,4.0,176.706842,106.0,4.0,27.157274,9.5,0.3,7.394926,98.0,19.0,67.656132,3552.0,4.0,126.002039,78.0,4.0,25.415598,9.3,2.5,7.515152,92.0,28.0,68.641884,528.000000,8.0,113.300000,53.000000,9.0,28.800000,9.1,5.5,8.045000,85.0,69.0,79.250000,10665.0,4.0,472.765416,107.0,4.0,38.224482,9.4,1.0,7.178311,98.0,13.0,71.652216,1932.000000,152.000000,672.000000,437.00000,42.000000,4.000000,0.547619,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,91.000000,17.000000,9.0,132.000000,0,0,0,0,100.000000,50.000000,57.982756,149.000000,74.500000,81.317280,1282.0,4.0,150.268968,26.360821,4.0,23.696523,9.4,1.2,7.808855,98.0,26.0,69.393296,8003.0,4.0,176.706842,106.0,4.0,27.157274,9.5,0.3,7.394926,98.0,19.0,67.656132,3552.0,4.0,126.002039,78.0,4.0,25.415598,9.3,2.5,7.515152,92.0,28.0,68.641884,1288.000000,6.0,102.458393,102.000000,6.0,33.108866,9.3,5.8,7.876000,91.0,65.0,75.517414,7064.0,4.0,116.653145,113.0,4.0,28.086735,9.7,0.5,7.356062,98.0,21.0,68.843507,1547.000000,1188.000000,819.000000,2244.00000,149.000000,-115.000000,0.114094,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
2,87.000000,28.000000,8.5,39.000000,0,0,0,0,95.500000,47.750

# tabnet

In [27]:
n_seeds = 1
n_splits = 5
shuffle = True

batch_size = 64
#batch_size = 8
factor = 0.5
patience = 30
lr = 0.001
#fit_params = {"epochs": 1_000, "verbose": 1}
fit_params = {"epochs": 300, "verbose": 1}

params = dict(
    epsilon=1e-05,
    feature_columns=None,
    virtual_batch_size=None,
    # num_layers=2,
    num_decision_steps=1,
    norm_type="batch",
    #norm_type="group",
    num_groups=-1,
    batch_momentum=0.9,
    relaxation_factor=1.2,
    sparsity_coefficient=0.0001,
    feature_dim=1024,
    output_dim=209,
)
with open(f"params.pkl", "wb") as f:
    pickle.dump(params, f)

#DEBUG = True
DEBUG = False
if DEBUG:
    n_seeds = 1
    n_splits = 2
    batch_size = 1024
    fit_params = {"epochs": 10, "verbose": 1}
    print("DEBUG")

In [28]:
## test setとあまり被りのない特徴量は除外  https://www.guruguru.science/competitions/13/discussions/df06ef19-981d-4666-a0c0-22f62ee26640/
#
#inbalance = ["Publisher", "Developer", "Name"]
#train_drop_sales = train_drop_sales.drop(inbalance, axis=1)
#test = test.drop(inbalance, axis=1)
#
#cate_cols = list(set(cate_cols) - set(inbalance))
#print("cate_cols:", cate_cols)

In [29]:
#_train = pd.read_csv(f"{DATADIR}/train.csv")

# group_col = "Name"
group_col = "Publisher"  # https://www.guruguru.science/competitions/13/discussions/42fc473d-4450-4cfc-b924-0a5d61fd0ca7/

# GroupKFold
group = train[group_col].copy()  

# seed値が指定できるGroupKFold  https://www.guruguru.science/competitions/13/discussions/cc7167cb-3627-448a-b9eb-7afcd29fd122/
group_uni = train[group_col].copy().unique()

In [30]:
features = test.columns.to_list()

global_target_col = "Global_Sales"
target_cols = [
    "NA_Sales",
    "EU_Sales",
    "JP_Sales",
    "Other_Sales",
]

X = train_drop_sales[features]
Y = train[target_cols]
Y_global = train[global_target_col]

train_size, n_features = X.shape
_, n_classes = Y.shape

In [31]:
def root_mean_squared_logarithmic_error(y_true, y_pred):
    msle = tf.keras.losses.MeanSquaredLogarithmicError()
    return K.sqrt(msle(y_true, y_pred))


def fit_tabnet(X_train, Y_train, X_val, Y_val, model_path):
    K.clear_session()
    model = TabNetRegressor(num_regressors=n_classes, num_features=n_features, **params)
    model.compile(
        optimizer=AdaBeliefOptimizer(learning_rate=lr), 
        #loss=tf.keras.losses.MeanSquaredLogarithmicError(),  # MSLE
        loss=root_mean_squared_logarithmic_error,  # RMSLE
    )
    
    callbacks = build_callbacks(model_path, factor=factor, patience=patience)
    
    history = model.fit(
        X_train, 
        Y_train,
        batch_size=batch_size,
        callbacks=callbacks,
        validation_data=(X_val, Y_val),
        **fit_params,
    )
    
    model.load_weights(model_path)
    
    return model

In [32]:
def fit_model(Y, features_imp=None, out_pkl="Y_pred.pkl"): 
    
    Y_pred = np.zeros((train_size, n_classes))
    Y_pred = pd.DataFrame(Y_pred, columns=Y.columns, index=Y.index)

    for i in tqdm(range(n_seeds)):
        set_seed(seed=i)

        cv = KFold(n_splits=n_splits, random_state=i, shuffle=shuffle)
        #cv_split = cv.split(X, Y)  # KFold
        cv_split = cv.split(group_uni)  # GroupRandomKFold
        #cv = GroupKFold(n_splits=n_splits)
        #cv_split = cv.split(X, Y, group)

        for j, (trn_idx, val_idx) in enumerate(cv_split):        
            print(f"\n------------ fold:{j} ------------")
            # GroupRandomKFold
            tr_groups, va_groups = group_uni[trn_idx], group_uni[val_idx]
            trn_idx = group[group.isin(tr_groups)].index
            val_idx = group[group.isin(va_groups)].index
            print("len(trn_idx), len(val_idx):", len(trn_idx), len(val_idx))

            X_train, X_val = X.iloc[trn_idx], X.iloc[val_idx]
            Y_train, Y_val = Y.iloc[trn_idx], Y.iloc[val_idx]

            # importance 上位100位までの特徴量だけにする
            if features_imp is not None:
                X_train, X_val = X_train[features_imp], X_val[features_imp]
                
            model = fit_tabnet(X_train, Y_train, X_val, Y_val, f"model_seed_{i}_fold_{j}.h5")
            
            va_pred = model.predict(X_val)
            
            va_pred = np.where(va_pred<0, 0, va_pred)
            
            Y_pred.iloc[val_idx] += va_pred / n_seeds
            
            #del model
            #gc.collect()

    with open(out_pkl, "wb") as f:
        pickle.dump(Y_pred, f)

    return Y_pred, model

In [33]:
%%time
Y_pred, model = fit_model(Y)


------------ fold:0 ------------
len(trn_idx), len(val_idx): 6174 2185
Please check your arguments if you have upgraded adabelief-tf from version 0.0.1.
Modifications to default arguments:
                           eps  weight_decouple    rectify
-----------------------  -----  -----------------  -------------
adabelief-tf=0.0.1       1e-08  Not supported      Not supported
Current version (0.1.0)  1e-14  supported          default: True
For a complete table of recommended hyperparameters, see
https://github.com/juntang-zhuang/Adabelief-Optimizer

Epoch 1/300

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocastin

118/118 [==============================] - 1s 11ms/step - loss: 1.4658 - val_loss: 0.9178
Epoch 2/300
118/118 [==============================] - 1s 9ms/step - loss: 1.2606 - val_loss: 0.8985
Epoch 3/300
118/118 [==============================] - 1s 9ms/step - loss: 1.2184 - val_loss: 0.8709
Epoch 4/300
118/118 [==============================] - 1s 9ms/step - loss: 1.1897 - val_loss: 0.8559
Epoch 5/300
118/118 [==============================] - 1s 9ms/step - loss: 1.1756 - val_loss: 0.8938
Epoch 6/300
118/118 [==============================] - 1s 9ms/step - loss: 1.1603 - val_loss: 0.8663
Epoch 7/300
118/118 [==============================] - 1s 9ms/step - loss: 1.1523 - val_loss: 0.8482
Epoch 8/300
118/118 [==============================] - 1s 9ms/step - loss: 1.1293 - val_loss: 0.8934
Epoch 9/300
118/118 [==============================] - 1s 9ms/step - loss: 1.1272 - val_loss: 0.8195
Epoch 10/300
118/118 [==============================] - 1s 9ms/step - loss: 1.1133 - val_loss: 0.8347


91/91 [==============================] - 1s 10ms/step - loss: 1.0575 - val_loss: 1.1819
Epoch 21/300
91/91 [==============================] - 1s 10ms/step - loss: 1.0573 - val_loss: 1.1323
Epoch 22/300
91/91 [==============================] - 1s 9ms/step - loss: 1.0608 - val_loss: 1.1949
Epoch 23/300
91/91 [==============================] - 1s 10ms/step - loss: 1.0549 - val_loss: 1.1133
Epoch 24/300
91/91 [==============================] - 1s 9ms/step - loss: 1.0432 - val_loss: 1.1204
Epoch 25/300
91/91 [==============================] - 1s 9ms/step - loss: 1.0479 - val_loss: 1.2229
Epoch 26/300
91/91 [==============================] - 1s 9ms/step - loss: 1.0367 - val_loss: 1.1225
Epoch 27/300
91/91 [==============================] - 1s 9ms/step - loss: 1.0399 - val_loss: 1.1450
Epoch 28/300
91/91 [==============================] - 1s 9ms/step - loss: 1.0388 - val_loss: 1.1407
Epoch 29/300
91/91 [==============================] - 1s 9ms/step - loss: 1.0352 - val_loss: 1.1980
Epoch 30/3

108/108 [==============================] - 1s 9ms/step - loss: 1.0349 - val_loss: 1.1842
Epoch 27/300
108/108 [==============================] - 1s 9ms/step - loss: 1.0298 - val_loss: 1.1145
Epoch 28/300
108/108 [==============================] - 1s 9ms/step - loss: 1.0293 - val_loss: 1.2033
Epoch 29/300
108/108 [==============================] - 1s 9ms/step - loss: 1.0086 - val_loss: 1.0928
Epoch 30/300
108/108 [==============================] - 1s 9ms/step - loss: 0.9984 - val_loss: 1.1787
Epoch 31/300
108/108 [==============================] - 1s 9ms/step - loss: 0.9989 - val_loss: 1.1998
Epoch 32/300
108/108 [==============================] - 1s 9ms/step - loss: 0.9998 - val_loss: 1.1499
Epoch 33/300
108/108 [==============================] - 1s 9ms/step - loss: 1.0001 - val_loss: 1.1588
Epoch 34/300
108/108 [==============================] - 1s 9ms/step - loss: 0.9950 - val_loss: 1.1312
Epoch 35/300
108/108 [==============================] - 1s 9ms/step - loss: 0.9925 - val_loss: 

111/111 [==============================] - 1s 9ms/step - loss: 0.9581 - val_loss: 0.9731
Epoch 46/300
111/111 [==============================] - 1s 9ms/step - loss: 0.9589 - val_loss: 1.0179
Epoch 47/300
111/111 [==============================] - 1s 9ms/step - loss: 0.9625 - val_loss: 1.0020
Epoch 48/300
111/111 [==============================] - 1s 9ms/step - loss: 0.9570 - val_loss: 0.9793
Epoch 49/300
111/111 [==============================] - 1s 9ms/step - loss: 0.9568 - val_loss: 1.0154
Epoch 50/300
111/111 [==============================] - 1s 9ms/step - loss: 0.9542 - val_loss: 0.9693
Epoch 51/300
111/111 [==============================] - 1s 9ms/step - loss: 0.9486 - val_loss: 0.9678
Epoch 52/300
111/111 [==============================] - 1s 9ms/step - loss: 0.9454 - val_loss: 0.9602
Epoch 53/300
111/111 [==============================] - 1s 9ms/step - loss: 0.9466 - val_loss: 0.9745
Epoch 54/300
111/111 [==============================] - 1s 9ms/step - loss: 0.9301 - val_loss: 

In [34]:
score(Y, Y_pred)

NA_Sales       1.314835
EU_Sales       1.217597
JP_Sales       1.205844
Other_Sales    0.847330
dtype: float64

In [35]:
# デフォルトパラメ
## RankGaussなし: 1.18526
## RankGaussあり: 1.23384
# パラメ変えて
## bauch_size=8: 1.16616
## bauch_size=64: 1.14625
score(Y_global, Y_pred.sum(axis=1))

1.146255058286068

# oof

In [36]:
path = r"Y_pred.pkl"
with open(path, "rb") as f:
    Y_pred = pickle.load(f)
Y_pred

,NA_Sales,EU_Sales,JP_Sales,Other_Sales
0,17.294683,6.585598,0.567128,3.727135
1,36.258926,15.381187,1.361468,8.673420
2,10.828993,3.831598,0.876301,1.860105
3,18.843252,7.936788,1.044856,2.325323
4,15.814473,6.157887,0.478688,3.589277
...,...,...,...,...
8354,2.009439,12.884875,0.264278,3.090280
8355,1.033238,4.692636,0.000000,0.819955
8356,0.624857,0.131230,1.480749,0.162746
8357,6.802473,7.420416,2.153712,3.286767


# predict test

In [37]:
Y_test_pred = np.zeros((test.shape[0], n_classes))
Y_test_pred = pd.DataFrame(Y_test_pred, columns=target_cols, index=test.index)

for i in range(n_seeds):
    for j in range(n_splits):
        model_path = f"model_seed_{i}_fold_{j}.h5"
        model = TabNetRegressor(num_regressors=n_classes, num_features=n_features, **params)
        model(np.zeros((1, n_features)))
        model.load_weights(model_path)
            
        Y_test_pred += model.predict(test) / (n_seeds * n_splits)
            
for col in Y_test_pred.columns:
    idx = Y_test_pred[Y_test_pred[col] < 0].index
    Y_test_pred[col][idx] = 0.0
    
print(Y_test_pred.shape)

submission = pd.read_csv(f"{DATADIR}/atmaCup8_sample-submission.csv")
submission["Global_Sales"] = Y_test_pred
submission.to_csv("submission.csv", index=False)
submission


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.


If you intended

,Global_Sales
0,13.087490
1,31.999891
2,18.447356
3,29.122060
4,22.518407
...,...
8355,11.180462
8356,4.882340
8357,1.346418
8358,1.541184
